In [1]:
import pandas as pd

In [70]:
from keras.layers import Input, Dense
from snn import SNN, Entity, Relation

In [71]:
Person = Entity("Person")
Movie = Entity("Movie")

In [104]:
df = pd.read_csv("../data/relations.tsv", sep="\t", names=['Movie', 'Relation', 'Person'], nrows=100000)

In [105]:
df.head()

,Movie,Relation,Person
0,Carmencita,self,Carmencita
1,Carmencita,director,William K.L. Dickson
2,Carmencita,cinematographer,William Heise
3,Le clown et ses chiens,director,Émile Reynaud
4,Le clown et ses chiens,composer,Gaston Paulin


In [106]:
relations = {l: Relation(label=l, dst=Person, src=Movie) for l in df['Relation'].unique()}
relations

{'actor': actor(<Movie> , <Person>),
 'actress': actress(<Movie> , <Person>),
 'archive_footage': archive_footage(<Movie> , <Person>),
 'cinematographer': cinematographer(<Movie> , <Person>),
 'composer': composer(<Movie> , <Person>),
 'director': director(<Movie> , <Person>),
 'editor': editor(<Movie> , <Person>),
 'producer': producer(<Movie> , <Person>),
 'production_designer': production_designer(<Movie> , <Person>),
 'self': self(<Movie> , <Person>),
 'writer': writer(<Movie> , <Person>)}

In [107]:
entities = [Person, Movie]
entities

[<Person>, <Movie>]

In [108]:
X = [t.Person + " @ " + t.Movie for t in df.itertuples()]
y = [[relations[t.Relation]] for t in df.itertuples()]

In [109]:
from sklearn.feature_extraction.text import CountVectorizer

In [110]:
cv = CountVectorizer()
Xt = cv.fit_transform(X)

In [111]:
x = Input(shape=(Xt.shape[1],))
snn = SNN(entities=entities, relations=relations.values())
output = snn(x)

In [112]:
from snn.utils import draw

In [113]:
snn_model = snn.build(x)

In [114]:
yp = snn_model.predict(Xt)
yp = (yp > 0.5).astype(int)
yp

array([[0, 0, 1, ..., 1, 0, 0],
       [1, 0, 1, ..., 1, 0, 0],
       [1, 0, 1, ..., 1, 1, 0],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 1, 1]])

In [115]:
yt = snn.map(y)
yt

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [116]:
(yp == yt).mean()

0.5224269230769231

In [117]:
snn_model.fit(Xt, yt, epochs=10, validation_split=0.1)

Train on 90000 samples, validate on 10000 samples
Epoch 1/10
90000/90000 [==============================] - 66s 729us/step - loss: 0.1130 - val_loss: 0.1286
Epoch 2/10
90000/90000 [==============================] - 65s 718us/step - loss: 0.0835 - val_loss: 0.1309
Epoch 3/10
 4480/90000 [>.............................] - ETA: 59s - loss: 0.0731

KeyboardInterrupt: 

In [118]:
yp = snn_model.predict(Xt)
yp = (yp > 0.5).astype(int)
(yp == yt).mean()

0.9713315384615384